<a href="https://colab.research.google.com/github/WellingtonFidelis/ml-marketing-digital/blob/main/BigData_ML_MarketingDigital_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB 2

## Machine Learning em Marketing Digital

Prevendo número de usuários convertidos

## Carregando bibliotecas

In [1]:
# Se necessário instalar a biblioteca abaixo
# ! pip install tidyverse

In [2]:
import pandas as pd
from google.colab import drive
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import pickle

## Carregando o dataset

In [3]:
# Mount Google Drive
drive.mount('/content/drive')

# Specify the path to the dataset in your Google Drive
dataset_path = '/content/drive/MyDrive/Courses/BigData_Analytics_R_MS_A_ML/Cap11/Regressão/dataset.csv'

# Load the dataset using Pandas
df = pd.read_csv(dataset_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Pré-Analise

Primeiras linhas do dataset

In [4]:
df.head(n=5)

,valor_gasto_campanha,numero_visualizacoes,numero_cliques,usuarios_convertidos
0,875,4390,54,30
1,1451,7120,90,72
2,1232,6198,67,47
3,1099,5554,70,63
4,656,3336,41,34


Tipos de dados

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   valor_gasto_campanha  500 non-null    int64
 1   numero_visualizacoes  500 non-null    int64
 2   numero_cliques        500 non-null    int64
 3   usuarios_convertidos  500 non-null    int64
dtypes: int64(4)
memory usage: 15.8 KB


## Análise Exploratória

In [6]:
df.describe()

,valor_gasto_campanha,numero_visualizacoes,numero_cliques,usuarios_convertidos
count,500.000000,500.000000,500.000000,500.000000
mean,998.554000,5004.508000,64.568000,49.626000
std,298.688354,1487.477168,20.928195,18.405969
min,505.000000,2376.000000,9.000000,4.000000
25%,741.500000,3710.500000,47.000000,36.000000
50%,1013.000000,5112.500000,65.000000,49.000000
75%,1256.500000,6279.500000,81.250000,63.000000
max,1493.000000,7528.000000,116.000000,101.000000


In [7]:
df.shape

(500, 4)

**Heatmap**

Map de calor

In [8]:
fig = px.imshow(df.corr(), text_auto=True, title='Mapa de calor')

fig.show()

❗ Espera-se, sempre, o menor valor na correlação de variáveis preditoras.

⚠ Há um problema de multicolinearidade, logo há uma alta correlação entre as variáveis preditoras ou independentes.

Scatter plot entre valor gasto em campanha e usuários convertidos

In [9]:
px.scatter(
    data_frame=df,
    x=df['valor_gasto_campanha'],
    y=df['usuarios_convertidos'],
    # color=''
    title='Valor gasto em campanha versos usuários convertidos',
    labels={'usuarios_convertidos': 'Usuários convertidos', 'valor_gasto_campanha': 'Valor gasto'}
  )

Scatter plot entre número de visualizações e usuários convertidos

In [10]:
px.scatter(
    data_frame=df,
    x=df['numero_visualizacoes'],
    y=df['usuarios_convertidos'],
    # color=''
    title='Número de visualizações versos usuários convertidos',
    labels={'usuarios_convertidos': 'Usuários convertidos', 'numero_visualizacoes': 'Número de visualizações'}
  )

Scatter plot entre número de cliques e usuários convertidos

In [11]:
px.scatter(
    data_frame=df,
    x=df['numero_cliques'],
    y=df['usuarios_convertidos'],
    # color=''
    title='Número de cliques versos usuários convertidos',
    labels={'usuarios_convertidos': 'Usuários convertidos', 'numero_cliques': 'Número de cliques'}
  )

## Modelagem 1

### Versão 1

Regressão linear multipla

In [12]:
X=df[['numero_cliques','numero_visualizacoes','valor_gasto_campanha']]
y=df['usuarios_convertidos']
model1 = LinearRegression()
model1.fit(X=X, y=y)

LinearRegression()

**Coeficiente de determinação (R²)**

O Coeficiente de determinação (R²) indica o quanto, em porcentagem, o modelo consegue explicar 86% da variância da variável "target" com este modelo.
Quando mais próximo de 1 melhor.

In [13]:
model1.score(X=X, y=y)

0.8617331627711401

**Intercepto ou constante de modelo**

O intercepto é o ponto da reta que encontra o eixo Y. Ou seja, é o valor de Y quando x=0.

Seria o **'a'** da expressão Y=a+bx+*erro*

In [14]:
model1.intercept_

-1.5634634573276642

**Coeficientes ou parâmetros do modelo**

O coeficiente angular é o quanto a reta está inclinada. Determina se a reta será crescente ou decrescente.

Na expressão da regressão linear (Y=a+bx) é o valor do **'b'**.

❗ Como nesse modelo usamos mais de uma variável preditora/independente há mais de um coeficiente

In [15]:
model1.coef_

array([ 0.94388169, -0.00350504,  0.00779719])

In [16]:
model1.rank_

3

**Residuais**

Os resíduos são a diferença entre os valores "target", valores que queremos prever também chamados de valores observados e que também são os usados para avaliar o modelo, e os valores previstos pelo modelo treinado.

In [17]:
prediction = model1.predict(X)
residuals = y - prediction
residuals.mean()

1.4068746168049984e-15

In [18]:
fig = go.Figure(layout={'title':{'text':'Modelo 1: Valores previstos versus treino'}})
fig.update_xaxes(title_text='Quantidade de usuários convertidos')
fig.update_layout(legend_title_text='Legenda')
fig.add_trace(go.Scatter(x=list(range(0, 100)), y=df['usuarios_convertidos'],
                    mode='lines',
                    name='Train values',
                    hovertemplate='Train value: <b>%{y}</b>'))
fig.add_trace(go.Scatter(x=list(range(0, 100)), y=prediction,
                    mode='lines',
                    name='Predict values',
                    hovertemplate='Predict value: <b>%{y}</b>'))
#fig.add_trace(go.Scatter(x=random_x, y=random_y2,
#                    mode='markers', name='markers'))

fig.show()

### Versão 2

**Regressão linear simples**

In [19]:
X=df[['numero_cliques']]
y=df['usuarios_convertidos']
model2 = LinearRegression()
model2.fit(X=X, y=y)

LinearRegression()

**Coeficiente de determinação (R²)**

In [20]:
model2.score(X=X, y=y)

0.8585226652192677

**Intercepto ou constante de modelo**

In [21]:
model2.intercept_

-2.990277359986372

**Coeficientes *angular* ou parâmetros do modelo**

In [22]:
model2.coef_

array([0.81489712])

**Residuais**

In [23]:
prediction = model2.predict(X)
residuals = y - prediction
residuals.mean()

-1.5845103007450234e-15

In [24]:
fig = go.Figure(layout={'title':{'text':'Modelo 2: Valores previstos versus treino'}})
fig.update_xaxes(title_text='Quantidade de usuários convertidos')
fig.update_layout(legend_title_text='Legenda')
fig.add_trace(go.Scatter(x=list(range(0, 100)), y=df['usuarios_convertidos'],
                    mode='lines',
                    name='Train values',
                    hovertemplate='Train value: <b>%{y}</b>'))
fig.add_trace(go.Scatter(x=list(range(0, 100)), y=prediction,
                    mode='lines',
                    name='Predict values',
                    hovertemplate='Predict value: <b>%{y}</b>'))

fig.show()

## Resolvendo problema da multicolinearidade

Utilização de técnica de Feature Engineer

Criando nova variável chamada de taxa de cliques, que será igual ao número de cliques divido pelo numero de visualizações.

In [25]:
df['taxa_clique'] = df['numero_cliques'] / df['numero_visualizacoes']
df.head(n=5)

,valor_gasto_campanha,numero_visualizacoes,numero_cliques,usuarios_convertidos,taxa_clique
0,875,4390,54,30,0.012301
1,1451,7120,90,72,0.012640
2,1232,6198,67,47,0.010810
3,1099,5554,70,63,0.012604
4,656,3336,41,34,0.012290


Verificando se há algum valor zero na nova coluna criada.

In [26]:
(df['taxa_clique'] == 0).all()

False

**Heatmap**

In [27]:
fig = px.imshow(df.corr(), text_auto=True, title='Mapa de calor')
fig.show()

❗ Nota-se que a nova variável *'taxa_clique'* possui um valor muito menor em relação às outras variáveis preditoras. Logo, é recomendado o uso desta.

## Modelagem 2

**Regressão linear multipla**

In [28]:
X=df[['valor_gasto_campanha', 'taxa_clique']]
y=df['usuarios_convertidos']
model3 = LinearRegression()
model3.fit(X=X, y=y)

LinearRegression()

**Coeficiente de determinação (R²)**

In [29]:
model3.score(X=X, y=y)

0.8418018245872498

**Intercepto ou constante de modelo**

In [30]:
model3.intercept_

-47.85206472383286

**Coeficientes *angular* ou parâmetros do modelo**

In [31]:
model3.coef_

array([5.10469825e-02, 3.61258711e+03])

**Residuais**

In [32]:
prediction = model3.predict(X)
residuals = y - prediction
residuals.mean()

-6.94910795573378e-15

In [33]:
fig = go.Figure(layout={'title':{'text':'Modelo 3: Valores previstos versus treino'}})
fig.update_xaxes(title_text='Quantidade de usuários convertidos')
fig.update_layout(legend_title_text='Legenda')
fig.add_trace(go.Scatter(x=list(range(0, 100)), y=df['usuarios_convertidos'],
                    mode='lines',
                    name='Train values',
                    hovertemplate='Train value: <b>%{y}</b>'))
fig.add_trace(go.Scatter(x=list(range(0, 100)), y=prediction,
                    mode='lines',
                    name='Predict values',
                    hovertemplate='Predict value: <b>%{y}</b>'))

fig.show()

## Suposições

Residuais versus valores ajustados

In [34]:
fig = go.Figure()
fig.update_layout({'title':{'text':'Modelo 3: Residuos versus valores ajustados'}})
fig.update_yaxes(title_text='Residuos')
fig.update_xaxes(title_text='Valores ajustados')
fig.update_layout(legend_title_text='Legenda')
fig.add_trace(
    go.Scatter(
        x=prediction,
        y=residuals,
        mode='markers',
        # name='Train values',
        hovertemplate='Train value: <b>%{y}</b>',
        opacity=0.65,
        connectgaps=False,
        )
    )
fig.show()

⚠ Melhorar este gráfico ⚠

In [35]:
fig = go.Figure()
fig.update_layout({
    'title':'Histograma de Resíduos'
})
fig.add_trace(
    go.Histogram(
        x=residuals,
        )
)

**Salvando o modelo em disco**

In [36]:
# with open(file='model3.pkl', mode='wb') as file:
#  pickle.dump(obj=model3, file=file)

Carregando o modelo

In [37]:
# with open(file='model3.pkl', mode='rb') as file:
#  model_to_deploy = pickle.load(file)

## Deploy

Implementação/uso do modelo

Colhendo valores

In [38]:
valor_gasto_campanha = 3000
numero_visualizacoes = 5000
numero_cliques = 300

Criando um dataframe do pandas para os novos dados de entrada

In [39]:
new_data = pd.DataFrame(data={
    'valor_gasto_campanha': [valor_gasto_campanha],
    'numero_visualizacoes': [numero_visualizacoes],
    'numero_cliques': [numero_cliques]
})
new_data.head()

,valor_gasto_campanha,numero_visualizacoes,numero_cliques
0,3000,5000,300


Realizando o processo de Engenharia de atributos para termos as mesmas entradas que o modelo teve em seu treinamento.

**Toda e qualquer transformação aplicadas nos dados de treinos deve ser aplicado nos dados de teste e novos dados.**

In [40]:
new_data['taxa_clique'] = new_data['numero_cliques'] / new_data['numero_visualizacoes']
new_data.head()

,valor_gasto_campanha,numero_visualizacoes,numero_cliques,taxa_clique
0,3000,5000,300,0.06


In [41]:
new_data = new_data.drop(columns=['numero_cliques', 'numero_visualizacoes'])
new_data.head()

,valor_gasto_campanha,taxa_clique
0,3000,0.06


Realizando a predição

In [42]:
predict = model3.predict(X=new_data)
predict

array([322.04410935])

Ou seja, se a empresa investir R$ 3000,00 em campanha, tiver 5000 visualizações e tiver 300 cliques no anúncia, ou local específico, terá **322 usuários convertidos**.